# Time of layer 1. MNIST

In [1]:
import sys
sys.path.insert(1, '../../_tools/')
import torch as th
import torch.nn as nn
import numpy as np
import torchvision
import torchvision.transforms as trasf
from utils.tensordata import TDataset, TDataloader, PosNeg_Bootstrap_TDataloader
from ffa import FFA_withEntropy
import matplotlib.pyplot as plt

seed = 42
th.manual_seed(seed)
np.random.seed(seed)
device = th.device('cuda' if th.cuda.is_available() else 'cpu')

### Data

In [2]:
DS_torch_dataset = torchvision.datasets.MNIST(
    root=r"C:\Users\matte\LocalData\Master Thesis",
    train=True,
    download=True,
    transform=trasf.ToTensor()
)
TS_torch_dataset = torchvision.datasets.MNIST(
    root=r"C:\Users\matte\LocalData\Master Thesis",
    train=False,
    download=True,
    transform=trasf.ToTensor()
)

# Load everything
DS = TDataset(
    x=th.stack([x.flatten() for x, y in DS_torch_dataset]).to(device),
    y=th.tensor([y for x, y in DS_torch_dataset], device=device).reshape(-1, 1)
)
TS = TDataset(
    x=th.stack([x.flatten() for x, y in TS_torch_dataset]).to(device),
    y=th.tensor([y for x, y in TS_torch_dataset], device=device).reshape(-1, 1)
)
TR, VL = DS.random_split(5/6)

# Dataloaders
batch_size = 128
DS_dl = DS.dataloader(batch_size=batch_size)
TR_dl = TR.dataloader(batch_size=batch_size)
VL_dl = VL.dataloader(batch_size=batch_size)
TS_dl = TS.dataloader(batch_size=batch_size)
TR_pndl = PosNeg_Bootstrap_TDataloader(TR, batch_size=batch_size)
DS_pndl = PosNeg_Bootstrap_TDataloader(DS, batch_size=batch_size)

### Model

In [3]:
Model = FFA_withEntropy

***

Final models

In [4]:
hyp_Toff20 = {
    'Nclasses': 10,
    'A': (784+10, 20),
    'f_hid': nn.ReLU(),
    'lr_hot': 0.01,
    'lr_cold': 0.01,
    'momentum': 0.99, 
    'weight_decay': 0.0001,
    'temperature': 0,
    'kernel_scale': 10., # the value found in 'T on' experiment, to compare the value of H
    'Nepochs': 24
}
hyp_Toff200 = {
    'Nclasses': 10,
    'A': (784+10, 200),
    'f_hid': nn.ReLU(),
    'lr_hot': 0.05,
    'lr_cold': 0.05,
    'momentum': 0.99, 
    'weight_decay': 1e-8,
    'temperature': 0,
    'kernel_scale': 90., # the value found in 'T on' experiment, to compare the value of H
    'Nepochs': 24
}
hyp_Toff2000 = {
    'Nclasses': 10,
    'A': (784+10, 2000),
    'f_hid': nn.ReLU(),
    'lr_hot': 0.1,
    'lr_cold': 0.1,
    'momentum': 0.995, 
    'weight_decay': 1e-8,
    'temperature': 0,
    'kernel_scale': 900., # the value found in 'T on' experiment, to compare the value of H
    'Nepochs': 30
}
hyp_Ton20 = {
    'Nclasses': 10,
    'A': (784+10, 20),
    'f_hid': nn.ReLU(),
    'lr_hot': 0.01,
    'lr_cold': 0.01,
    'momentum': 0.99, 
    'weight_decay': 0.0001,
    'temperature': 0.1/3,
    'kernel_scale': 10.,
    'Nepochs': 24
}
hyp_Ton200 = {
    'Nclasses': 10,
    'A': (784+10, 200),
    'f_hid': nn.ReLU(),
    'lr_hot': 0.05,
    'lr_cold': 0.05,
    'momentum': 0.99, 
    'weight_decay': 1e-6,
    'temperature': 3,
    'kernel_scale': 90.,
    'Nepochs': 24
}
hyp_Ton2000 = {
    'Nclasses': 10,
    'A': (784+10, 2000),
    'f_hid': nn.ReLU(),
    'lr_hot': 0.1,
    'lr_cold': 0.1,
    'momentum': 0.995, 
    'weight_decay': 1e-8,
    'temperature': 30.,
    'kernel_scale': 900.,
    'Nepochs': 30
}

exps = {
    'Toff20': hyp_Toff20,
    'Toff200': hyp_Toff200,
    'Toff2000': hyp_Toff2000,
    'Ton20': hyp_Ton20,
    'Ton200': hyp_Ton200,
    'Ton2000': hyp_Ton2000,
}

***

In [5]:
import time
_tic = 0.

def tic ():
    global _tic
    _tic = time.time()
       
def toc ():
    measure = time.time() - _tic
    return measure

for exp, hyp in exps.items():
    print(exp, end=': ')
    times = []
    for trial in range(5):
        m = Model(hyp).to(device)
        tic()
        m.fit(DS_pndl)
        times.append(toc())
    mean = np.mean(times)
    std = np.std(times, ddof=1)
    print(f"{mean:.6g} +- {std:.3g} sec")

Toff20: 287.391 +- 19.7 sec
Toff200: 287.876 +- 5.47 sec
Toff2000: 356.384 +- 6.99 sec
Ton20: 343.27 +- 7.15 sec
Ton200: 345.051 +- 8.43 sec
Ton2000: 509.672 +- 7.28 sec
